# TRẦN ĐỖ TRUNG KIÊN – 52000679
# HUỲNH HỒNG SƠN - 52000795
# NGUYỄN THÀNH HUY - 52100895

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving diabetes_012_health_indicators_BRFSS2021.csv to diabetes_012_health_indicators_BRFSS2021.csv


In [ ]:
!ls

diabetes_012_health_indicators_BRFSS2021.csv  sample_data


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf

In [ ]:
data = pd.read_csv('diabetes_012_health_indicators_BRFSS2021.csv')

print(data.head())

# Kiểm tra các missing value
print(data.isnull().sum())

   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0       0       1.0          1  15.0     1.0     0.0   
1           2.0       1       0.0          1  28.0     0.0     0.0   
2           2.0       1       1.0          1  33.0     0.0     0.0   
3           2.0       0       1.0          1  29.0     0.0     1.0   
4           0.0       0       0.0          1  24.0     1.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0             0       1  ...              1   
1                   1.0             0       1  ...              1   
2                   0.0             1       1  ...              1   
3                   1.0             1       1  ...              1   
4                   0.0             0       0  ...              1   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex  Age  Education  \
0          0.0      5.0      10.0      20.0       0.0    0   11        4.0   
1       

In [ ]:
# Tách cột bệnh lý tiểu đường ra khỏi df
X = data.drop('Diabetes_012', axis=1)
y = data['Diabetes_012']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tối ưu hóa LSTM
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# Loại bỏ dummy variable trap
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')  # Assuming 2 classes for diabetes: 0 or 1
])

# model tuan tu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
4728/4728 [==============================] - 46s 9ms/step - loss: 0.4420 - accuracy: 0.8366 - val_loss: 0.4310 - val_accuracy: 0.8378
Epoch 2/50
4728/4728 [==============================] - 38s 8ms/step - loss: 0.4329 - accuracy: 0.8375 - val_loss: 0.4315 - val_accuracy: 0.8369
Epoch 3/50
4728/4728 [==============================] - 40s 8ms/step - loss: 0.4312 - accuracy: 0.8378 - val_loss: 0.4258 - val_accuracy: 0.8403
Epoch 4/50
4728/4728 [==============================] - 42s 9ms/step - loss: 0.4297 - accuracy: 0.8385 - val_loss: 0.4262 - val_accuracy: 0.8407
Epoch 5/50
4728/4728 [==============================] - 40s 8ms/step - loss: 0.4294 - accuracy: 0.8385 - val_loss: 0.4261 - val_accuracy: 0.8385
Epoch 6/50
4728/4728 [==============================] - 40s 8ms/step - loss: 0.4289 - accuracy: 0.8383 - val_loss: 0.4260 - val_accuracy: 0.8383
Epoch 7/50
4728/4728 [==============================] - 39s 8ms/step - loss: 0.4281 - accuracy: 0.8387 - val_loss: 0.4253 - val_ac

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

# Dự đoán từ dữ liệu train
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Report
print(classification_report(y_true, y_pred_classes))

1478/1478 [==============================] - 10s 7ms/step - loss: 0.4046 - accuracy: 0.8454
Accuracy: 84.54%
1478/1478 [==============================] - 8s 5ms/step
              precision    recall  f1-score   support

           0       0.86      0.98      0.91     39508
           1       1.00      0.00      0.00      1079
           2       0.59      0.18      0.28      6689

    accuracy                           0.85     47276
   macro avg       0.82      0.39      0.40     47276
weighted avg       0.82      0.85      0.80     47276



Mô hình này có loss và accuracy biến động không nhiều nên mô hình không bị overfitting